## Домашняя работа № 1

Работу выполнила Камская Милена, БКЛ-211

Для начала сделаем все нужные импорты

In [1]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import csv
import pandas as pd
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Имя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Пропишем функцию collect_reviews, которая на вход получает номер страницы и словарь, в который мы хотим вписать данные. На выходе данные в нашем словаре будут храниться так: ключом будет строка с текстом отзыва, а значением будет тональность.

In [2]:
def collect_reviews(page_number, dictionary):
    # скачиваем
    url = f'https://teatrow.ru/msk/moskovskij-dvorec-molodezhi/page,1,{page_number},16857-shahmaty.html'
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # находим все отзывы на странице и обрабатываем
    for review in soup.find_all('div', {'class': 'element-item 1'}):
        review_text = review.find('div', {'class': 'text mytext'}).text
        review_rating = int(review.find('meta', {'itemprop': 'ratingValue'})['content'])
        if review_rating == 4 or review_rating == 5:
            dictionary[review_text] = 'Positive'
        if review_rating == 1 or review_rating == 2 or review_rating == 3:
            dictionary[review_text] = 'Negative'
      
    return dictionary

Во-первых, я брала отзывы только на один определенный спектакль, чтобы не смешивать отзывы и уже повысить точность своей будущей модели. Грубо говоря, не мешала отзывы на кроссовки с отзывами на ноутбуки

Во-вторых, я ввела свой критерий измерения тональности. Если у отзывы 4 или 5 звезд, то я считала отзыв положительным, потому что есть некоторые гениальные диванные критики, которые 5 никому и никогда не ставят, а в отзыве пишут, что все понравилось. Мне важнее текст отзыва, чем звездочки. Отрицательными отзывами я считала отзывы с 1ой, 2мя или 3мя звездами.

In [3]:
dictionary_of_reviews = {}
for i in range(1, 101):
    collect_reviews(i, dictionary_of_reviews)

После того, как мы запустили функцию, мы смотрим на то, сколько у нас положительных и отрицательных отзывов с помощью Counter

In [4]:
c = Counter(dictionary_of_reviews.values())
c

Counter({'Negative': 150, 'Positive': 847})

Видим огромную разницу, но не отчаиваемся. Сортируем словарь по значению так, что первыми будут негативне отзывы, а после блока негатива будет блок позитива! И обрезаем, оставив 300 отзывов (150 отрицательных и 150 положительных). Так мы и получили информацию, которая позже станет обучающей выборкой

In [5]:
s = sorted(dictionary_of_reviews.items(), key=lambda item: item[1])
s = s[:300]

Теперь все сохраняем в csv, потому что так хранить удобнее)

In [6]:
def save(info):
    with open('reviews_train.csv','w', encoding='UTF-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Text','Sentiment'])

        for i in range(0,300):
            writer.writerow([info[i][0], info[i][1]])

save(s)

Теперь нехитрым способом получаем будущую тестовую выборку и тоже сохраняем ее в csv

In [7]:
dictionary_of_reviews_test = {}
collect_reviews(107, dictionary_of_reviews_test) #107 страница потому что тут есть и положительный и отрицательный отзывы и их ровно 10)
n = sorted(dictionary_of_reviews_test.items(), key=lambda item: item[1])

with open('reviews_test.csv','w', encoding='UTF-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Text','Sentiment'])
        for i in range(0,10):
              writer.writerow([n[i][0], n[i][1]])

Переходим ко второму пункту домашнего задания. Я прописала все функциями и затем просто их применю к данным

In [8]:
def convert_lowercase(text): #приводим все к нижнему регистру
    text = text.lower()
    return text

def remove_punc(text): #убираем пунктуацию
    return re.sub(r'[^\w\s]', ' ', text)

def remove_stopwords(text):
    new_list = []
    text = re.sub(r'[0-9]+', '', text) #убираем числа
    words = word_tokenize(text)
    stopwrds = stopwords.words('russian')
    for word in words:
        if word not in stopwrds:
            new_list.append(word)
    return ' '.join(new_list)

morph = pymorphy2.MorphAnalyzer()

def f_form(text): #приводим все слова к начальной форме
    lst = []
    for word in text.split(' '):
        w = morph.parse(word)[0]
        wrd = w.normal_form
        lst.append(wrd)
    return ' '.join(lst)
            

Прописываем функцию, которая на вход получает файл, а на выходе у нас полностью обработанный датасет в csv

In [9]:
def preparation(fname):
    data = pd.read_csv(fname,sep=',', encoding='UTF-8')
    data.columns = data.columns.str.replace('"', '')
    data['Text'] = data['Text'].astype(str) 
    data['Text'] = data['Text'].apply(convert_lowercase)
    data['Text'] = data['Text'].apply(remove_punc)
    data['Text'] = data['Text'].apply(remove_stopwords)
    data['Text'] = data['Text'].apply(f_form)
    data.to_csv(fname, index=False)

In [10]:
preparation('reviews_train.csv')

Теперь создадим 2 множества: в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных.

In [11]:
data = pd.read_csv('reviews_train.csv',sep=',', encoding='UTF-8')

text_positive = data[data['Sentiment'] == 'Positive']['Text']
text_negative = data[data['Sentiment'] == 'Negative']['Text']

In [12]:
def dict_for_set(text): #функция для словаря отдельная, потому что он нам потом понадобится (спойлер)
    all = []
    for line in text:
        w = line.split(' ')
        for word in w:
            all.append(word)

    d = {}
    for item in set(all):
        num = all.count(item)
        d[item] = num

    pure_d = {}
    for k, v in d.items():
        if v > 10: #здесь те слова, которые встречаются менее 11 раз я убираю, потому что они запутают модель
            pure_d[k] = v

    return pure_d

def set_of_words(text):
    d = dict_for_set(text)
    
    return set(d.keys())

А вот и эти множества!

In [13]:
s_pos = set_of_words(text_positive)
s_neg = set_of_words(text_negative)

Теперь создадим функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём

In [14]:
freq_lists = {}
freq_lists['Positive'] = dict_for_set(text_positive)
freq_lists['Negative'] = dict_for_set(text_negative)

А вот и сама функция. Она просто считает совпадения)

In [15]:
def simple_sent_detect(freq_lists, text):
    counts = Counter()
    for sent, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[sent] += int(freq_list[word] > 0)
    return counts.most_common()

Теперь нужно проверить, насколько у нас точный результат. Обработаем тестовые данные и представим их в виде словаря для того, чтобы было в дальнейшем удобнее к ним обращаться

In [16]:
preparation('reviews_test.csv')

In [17]:
data_test = open('reviews_test.csv', encoding='UTF-8')

In [18]:
test_dictionary = {}

for line in csv.reader(data_test):
    test_dictionary[line[0]] = line[1]

test_dictionary.pop('Text')
test_dictionary

{'понравиться постановка шумно внутренний стержень ссср суметь артист передать глубина душа': 'Negative',
 'удивить восторженный отзыв мюзикл собственно купиться понравиться муж сюжет очень политизировать сценарий толком прописать понятный смысл логика поступок герой откровенный дыра сценарий поэтому главный герой вызывать симпатия страдание выглядеть место комично напоминать индийский кино вообще очень затянутый мюзикл практически речь один ария бесконечно один очень хватать проникновенность игра актёр один кривляние главный героиня бесконечно хлопотать лицо тело бегать сцена создавать впечатление нервный экзальтированный дама который постоянно орать один ария запасть душа запомниться очень слабый перевод текст песня просто набор слово рифма петь слово просто говорить понравиться клишировать представление ссср советский народ типично иностранный взгляд шапка мишка балалайка кгб человек серый неуклюжий костюм вытянутый струнка выглядеть жалко фон американец пить водка стакан называть д

Выводим что нам дала модель. 

In [19]:
for line, sent in test_dictionary.items():
    print(simple_sent_detect(freq_lists, line)[0][0])

Positive
Negative
Negative
Negative
Positive
Positive
Negative
Positive
Positive
Positive


Круто, что работает, но как точно оно работает? Сейчас проверим!

Функция проверки на точность по сути просто сравнивает 2 списка (полученных результатов и ожиданий) при помощи sklearn

In [20]:
def test_simple_sent_detect(freq_lists, dictionary):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходную тональность
    for line, sent in test_dictionary.items():
        predicted_lang = simple_sent_detect(freq_lists, line)[0][0]
        results.append(predicted_lang)
        gold.append(sent)
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [21]:
test_simple_sent_detect(freq_lists, test_dictionary)

RESULTS:
Accuracy: 0.6000


Хм, как-то грустнено видеть 0.6, но модель достаточно простая. Все же неплохой результат

Теперь попробуем его улучшить

### Первое улучшение: умная фильтрация отзывов

Отзывы с тремя звездами вызывают сомнение, давайте посмотрим на них внимательнее

In [22]:
def collect_q_reviews(page_number, dictionary):
    # скачиваем
    url = f'https://teatrow.ru/msk/moskovskij-dvorec-molodezhi/page,1,{page_number},16857-shahmaty.html'
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # находим все отзывы на странице и обрабатываем
    for review in soup.find_all('div', {'class': 'element-item 1'}):
        review_text = review.find('div', {'class': 'text mytext'}).text
        review_rating = int(review.find('meta', {'itemprop': 'ratingValue'})['content'])
        if review_rating == 3:
            dictionary[review_text] = '???'
      
    return dictionary

In [23]:
dict_for_three = {}

for i in range(1, 21):
    collect_q_reviews(i, dict_for_three)

dict_for_three

{'Мюзикл не вызвал никаких эмоций. Относительно постановок, которые играются в культурной столице нашей страны, этот мюзикл не показался завораживающим. Думаю, мое мнение схоже с другими зрителями. Не было бурных оваций, все спешили поскорее покинуть зал. 3- / 5.': '???',
 'Искренне не понимаю, почему столько восторженных отзывов, каждому свое, конечно. Неплохая постановка, но, по моему мнению, в Москве есть гораздо более зрелищные мюзиклы и оперетты. По десятибальной шкале поставила бы 7': '???',
 'Разочарована! Совсем не зацепил, не запомнился! Вроде и артисты стараются, выкладываются, но … не случилось': '???',
 'К исполнителям и оркестру претензий нет - всё на высшем уровне. Приятно удивила Анастасия Стоцкая - не ожидал. Но музыка не зацепила, а к концу второго акта начал откровенно уставать. Особо отмечу Александра Маракулина - его партии это нечто.': '???',
 'Немного не хватило романтики... в целом интересная история': '???',
 'Нормально, но в рекомендацию не возьму. В МДМ очень 

Они спорные. В одном отзыве и положительная часть '*всё на высшем уровне*', и отрицательная '*Разочарована!*'. Чтобы улучшить качество модели я предлагаю убрать эти отзывы из выборки вообще. Сейчас будет большой код, повторение пройденного, но лучше!

In [24]:
def upgrade_collect_reviews(page_number, dictionary):
    # скачиваем
    url = f'https://teatrow.ru/msk/moskovskij-dvorec-molodezhi/page,1,{page_number},16857-shahmaty.html'
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # находим все отзывы на странице и обрабатываем
    for review in soup.find_all('div', {'class': 'element-item 1'}):
        review_text = review.find('div', {'class': 'text mytext'}).text
        review_rating = int(review.find('meta', {'itemprop': 'ratingValue'})['content'])
        if review_rating == 4 or review_rating == 5:
            dictionary[review_text] = 'Positive'
        if review_rating == 1 or review_rating == 2:
            dictionary[review_text] = 'Negative'
      
    return dictionary

In [25]:
upgrade_dictionary_of_reviews = {}
for i in range(1, 101):
    upgrade_collect_reviews(i, upgrade_dictionary_of_reviews)

In [26]:
Counter(upgrade_dictionary_of_reviews.values())

Counter({'Negative': 83, 'Positive': 847})

In [27]:
s = sorted(upgrade_dictionary_of_reviews.items(), key=lambda item: item[1])
s = s[:166]

In [28]:
def save(info):
    with open('reviews_train.csv','w', encoding='UTF-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Text','Sentiment'])

        for i in range(0,166):
            writer.writerow([info[i][0], info[i][1]])

save(s)

In [29]:
preparation('reviews_train.csv')

In [30]:
upgrade_data = pd.read_csv('reviews_train.csv',sep=',', encoding='UTF-8')

upgrade_text_positive = upgrade_data[upgrade_data['Sentiment'] == 'Positive']['Text']
upgrade_text_negative = upgrade_data[upgrade_data['Sentiment'] == 'Negative']['Text']

In [31]:
us_pos = set_of_words(upgrade_text_positive)
us_neg = set_of_words(upgrade_text_negative)

In [32]:
upgrade_freq_lists = {}
upgrade_freq_lists['Positive'] = dict_for_set(upgrade_text_positive)
upgrade_freq_lists['Negative'] = dict_for_set(upgrade_text_negative)

In [33]:
for line, sent in test_dictionary.items():
    print(simple_sent_detect(upgrade_freq_lists, line)[0][0])

Positive
Negative
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive


In [34]:
test_simple_sent_detect(upgrade_freq_lists, test_dictionary)

RESULTS:
Accuracy: 0.8000


Вау! Теперь наша точность 0.8 и это очень круто! Я подняла точность на 0.2, а это очень крутой результат

### Улучшение улучшений: TF-IDF

Я предлагаю на этапе формирования 2х множеств слов делать это через tf-idf. Тогда каждому слову присвоится его вес, который должен помочь при определении тональности и улучшить результаты.

In [35]:
data_train = pd.read_csv('reviews_train.csv', sep=',', encoding='UTF-8')
list_of_reviews = list(data_train['Text'])

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import wordpunct_tokenize

tfidf = TfidfVectorizer(
    analyzer = 'word',
    stop_words = stopwords.words('russian')
)

review_tfidf = tfidf.fit_transform(list_of_reviews)

Tfidf_data = pd.DataFrame(review_tfidf.toarray(), columns=tfidf.get_feature_names_out())

dict_neg = {}
try:
    for i in upgrade_freq_lists['Negative']:
        dict_neg[i] = Tfidf_data[i].sum()
except KeyError:
    print('error')
    
dict_pos = {}
try:
    for i in upgrade_freq_lists['Positive']:
        dict_pos[i] = Tfidf_data[i].sum()
except KeyError:
    print('error')

dict_tfidf = {}
dict_tfidf['Negative'] = dict_neg
dict_tfidf['Positive'] = dict_pos

error
error


In [37]:
def tfidf_sent_detect(dict_tfidf, text):
    counts = Counter()
    for sent, weight in dict_tfidf.items():
        weight = Counter(weight)
        for word in nltk.word_tokenize(text):
            counts[sent] += int(weight[word] > 0)
    return counts.most_common()

In [38]:
for line, sent in test_dictionary.items():
    print(tfidf_sent_detect(dict_tfidf, line)[0][0])

Positive
Negative
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive


In [39]:
def test_tfidf_sent_detect(dict_tfidf, dictionary):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходную тональность
    for line, sent in test_dictionary.items():
        predicted_lang = tfidf_sent_detect(dict_tfidf, line)[0][0]
        results.append(predicted_lang)
        gold.append(sent)
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [40]:
test_tfidf_sent_detect(dict_tfidf, test_dictionary)

RESULTS:
Accuracy: 0.8000


Почему он не улучшил? Я могу предположить, что это произошло из-за того, что я хорошо фильтровала учебную выборку, убирала редко встречающеся слова из freqlist и не брала спорные отзывы. Радует только одно, этот метод не ухудшил точность, а значит все хорошо и модель круто работает)
